# Setup

To run this demo, a file named `secret.py` must be created in the notebook's
folder with the following content:

```python
# Configuration settings

# SLIPO workbench installation
BASE_URL = 'https://app.dev.slipo.eu'

# SLIPO API key
API_KEY = 'your API key'
```

The `API_KEY` value must be set to a valid SLIPO Application Key. The file must be imported before creating a new context:

```python
from secret import BASE_URL, API_KEY
```

### Full documentation about the SLIPO API client is available [here](https://slipoframes.readthedocs.io/en/latest/).

#### Create new context for this session:

In [ ]:
from slipoframes.context import SlipoContext

from secret import BASE_URL, API_KEY

ctx = SlipoContext(
    base_url = BASE_URL,
    requires_ssl = False,
    api_key = API_KEY
)

#### Display all preconfigured **profiles** available per SLIPO Toolkit component:

In [ ]:
prof = ctx.profiles()
prof.sort_values(by=['Tool'])

# Execute an existing workflow

#### Run a prespecified data integration workflow that involves all stages (transformation, interlinking, fusion, enrichment, export).

Identify the workflow based on its user-specified _name_, its various versions and their executions:

In [ ]:
processes = ctx.process_query(
    'Integrate GET + OSM data for Corfu',
    0,
    10
)

exec = processes[['Id','Name','Executed On','Version']]
exec.sort_values(by=['Version'])

#### Execute a **new instance** of this workflow:

In [ ]:
workflow1 = ctx.process_save(1191)
workflow1 = ctx.process_start(workflow1)

#### Inspect the **status** of this workflow execution:

In [ ]:
workflow1 = ctx.process_status(workflow1)

#### Render this **workflow** as a graph with all its components and monitor its **progress**:

In [ ]:
ctx.process_render(workflow1)

# Execute successive stages in POI data integration

## Transform operation

Next, we are going to transform POI datasets into RDF, involving the following steps:

* Upload the files `get_pois_v02_corfu_2100.zip` (SHP format) and `osm20_pois_corfu.csv` (CSV format) from the local folder `datasets` to the remote folder `notebooks/datasets`. The remote folder will be created automatically if not already exists. The option `overwrite` is also set to `True` to overwrite any existing files.
* Execute two transform operations to convert data from `SHP` and `CSV` format into `N-Triples`.
* Check the status of each operation.

#### Upload **shapefile (SHP)** for the first input dataset (get_pois_v02_corfu_2100.zip) :

In [ ]:
ctx.file_upload('./datasets/get_pois_v02_corfu_2100.zip', 'notebooks/datasets/get_pois_v02_corfu_2100.zip', overwrite=True)

#### Upload **CSV file** for the second input dataset (osm20_pois_corfu.csv) :

In [ ]:
ctx.file_upload('./datasets/osm20_pois_corfu.csv', 'notebooks/datasets/osm20_pois_corfu.csv', overwrite=True)

#### Upload **configuration files** in folder `config` from the local file system to the remote folder `notebooks/config`:

In [ ]:
ctx.file_upload('./config', 'notebooks/config', overwrite=True)

#### Browse remote user file system to check its contents:

In [ ]:
df_files = ctx.file_browse(sort_col='size', format_size=True, sort_asc=False)

df_files[df_files['path'].str.startswith("notebooks")]

#### **Transformation # 1** for the first input dataset:

In [ ]:
# Convert SHP input file to N-Triples
transform1 = ctx.transform_shapefile(
    'notebooks/datasets/get_pois_v02_corfu_2100.zip',
    attrCategory='COMBOTYPE',
    attrKey='OBJECTID',
    attrName='NAME_GR',
    attrX='POINT_X',
    attrY='POINT_Y',
    featureSource='GET',
    sourceCRS='EPSG:2100',
    targetCRS='EPSG:4326',
    defaultLang='el',
    mappingSpec='notebooks/config/get_mappings.yml',
    classificationSpec='notebooks/config/get_pois_v04_poi_maincategories_en.csv'
)

#### **Transformation # 2** for the second input dataset:

In [ ]:
# Convert CSV input file to N-Triples
transform2 = ctx.transform_csv(
    'notebooks/datasets/osm20_pois_corfu.csv',
    attrCategory='type',
    attrGeometry='wkt',
    attrKey='osm_id',
    attrName='name',
    attrX='lon',
    attrY='lat',
    delimiter='|',
    featureSource='OpenStreetMap',
    profile='OSM_Europe',
    quote='',
    mappingSpec='notebooks/config/osm_mappings.yml',
    classificationSpec='notebooks/config/osm_classification.csv'
)

Check **progress status** of these transform operations:

In [ ]:
transform1 = ctx.process_status(transform1)
transform2 = ctx.process_status(transform2)

# Interlink operation

#### Execute an interlink operation on the **RDF datasets** generated by the previous two transformation operations:

In [ ]:
interlink1 = ctx.interlink(
    'SLIPO_equiMatchByNameAndDistance',
    left=transform1.output(),
    right=transform2.output()
)

#### Check **progress status** of the interlinking operation:

In [ ]:
interlink1 = ctx.process_status(interlink1)

# Fuse Operation

#### Fuse the **RDF datasets** generated by operations `transform1` and `transform2` using the **links** from operation `interlink`:

In [ ]:
fuse1 = ctx.fuse(
    'SLIPO_default_abMode',
    left=transform1.output(),
    right=transform2.output(),
    links=interlink1.output()
)

#### Check **progress status** of the fusion operation:

In [ ]:
fuse1 = ctx.process_status(fuse1)

# Enrich Operation

#### Enrich the fused RDF dataset returned from operation `fuse1` with extra properties from **DBpedia**:

In [ ]:
enrich1 = ctx.enrich(
    'SLIPO_BASIC',
    source=fuse1.output()
)

#### Check **progress status** of this enrichment operation:

In [ ]:
enrich1 = ctx.process_status(enrich1)

# Export Operation

#### Export the enriched RDF dataset to a **CSV file** for possible use in a database, a GIS application, analytics, etc:

In [ ]:
export1 = ctx.export_csv(
    'SLIPO_default',
    enrich1.output(),
    delimiter='|',
    quote='"',
    sparqlFile='notebooks/config/query.sparql'
)

#### Check **progress status** for export operation:

In [ ]:
export1 = ctx.process_status(export1)

#### Finally, copy output CSV file to the local file system (for **analytics** with Jupyter notebook):

In [ ]:
ctx.process_file_download(export1.output(), target='./output/corfu-integrated-pois.zip', overwrite=True)

#### Get integrated RDF triples as a local file:

In [ ]:
# Copy output RDFfile to local file system
ctx.process_file_download(enrich1.output(), target='./output/integrated.nt', overwrite=True)